# 前置知识

In [1]:
from IPython.display import HTML, display

# Reset CSS
display(HTML('<style>body {font-size: initial;}</style>'))


## k-foldCrossValidation Pattern
[F-fold 官方文档](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)

两种实现K-Fold交叉验证的方法,两者的区别在于是使用Sampler还是Subset

### 方法一：使用Range以及SubsetRandomSampler
具体实现就是TrainingTime中所做的实现

### 方法二：使用Range以及Subset
```python

# 实例化dataset对象
dataset = ImageDataset('train', path)

# 初始化KFold对象
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# 遍历每个fold
for fold, (train_idx, val_idx) in enumerate(kf.split(range(len(dataset)))):
    print(f'Fold {fold + 1}')
    
    # 通过索引创建训练和验证子集
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)
    
    # 创建DataLoader对象
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
```

# HW3
HW3要解决的问题是实物图像分类，这次HW比较有挑战性，如果要做到StrongBaseline或是BossBaseline,都需要用到  
许多没有了解的策略。比如Augmentation以及Testing，或是Training的技巧，测试上也可以用到集成的voting。以及  
将Augmentation用于Test Time Augmentation以及多模型预测结果的加权来保证测试结果的稳定性。设计上也要参考一   
些经典论文上提到的模型比如ResidualNet等  

先列一下大纲吧，要实现的东西有点多。  

1.数据处理
* `训练数据预处理`  由于网络结构中要用到CNN，图像根据固定输入大小如何预处理  
* `k-fold split`  将原来已近划分好的训练集验证集进行划分  
* `DataAugmentation`  由于模型可能构建的比较复杂，所以必须数据增强以避免过拟合。以及将数据增强方法用到测试集上

2.模型实现  （PreTrained Model NOT ALLOWED）
* `ResidualNet` 残差网络的使用  
* `CNN以及polling` CNN以及polling的使用(downSampling words better)  
* `SpatialTransform` ST层需要自己实现吗？  
* `ensemble` 实现并集成多模型的voting结果  
* `ClassificationTricks` Label smoothing Cross Entropy Loss、FocalLoss  
* `OptimTricks` Dropout、BatchNorm、GradientAccumulation、ImageNormalization

3.Training/Validation/Testing
* `Training` k-fold交叉验证实现思路：在最后训练时实现，定义kfold = KFold(n_splits=k_folds, shuffle=True)
枚举kfold.split(dataset)，之后利用torch.utils.data.SubsetRandomSampler初始化DataLoader就可以实现k-fold了
* `Validation`
* `Testing`

coding需要注意的点：　　
1.首先是编写网络架构经过每一层传递之后的张量大小有非常清楚的认识，主要是张量的　　　
长Ｈ 和宽Ｗ　因为这两个因素在residualNet中在shortcut之后会有残差和恒等映射的　　
相遇，这时候必须保证两个张量大小必须相同，但是习惯上对于identity映射中Ｈ和Ｗ不　　　
不对等的处理要么是下采样，要么是maxpool。下采样kernelsize＝１而在主路径中，　　　
kernelsize大小基本上不会是１这样就需要考虑padding大小的问题，如果选择了错误的　　　
padding就会导致两个张量大小不匹配　　

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader,ConcatDataset,Subset,SubsetRandomSampler
from torchvision.datasets import DatasetFolder,VisionDataset
import torchvision.transforms.v2 as v2
import torchvision.transforms.functional as F


#dataPreprocess
import numpy as np
import pandas as pd
import os
from PIL import Image

from tqdm.auto import tqdm
import random
from sklearn.model_selection import KFold

In [3]:
#图像变换的随机性主要由python内置的random模块决定，所以设定随即种子是有必要的
myseed = 420613
random.seed(myseed)
np.random.seed(myseed)
torch.backends.cudnn.derministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)


## Utilities

In [4]:
def get_device():
    return 'cuda:0' if torch.cuda.is_available() else 'cpu'

## Transforms/Augmentation

### 对于数据增强对模型训练效果的思考
使用数据增强来训练模型，很容易让人产生下面思考，只考虑一个epoch：
* 使用下面的tfm结构，一张图片在训练过程中(原图 | 增强图片)是不可能同时存在的
* 对于不同的图片(whether same label or not)只使用一个版本是否会造成模型学习效果不佳
* 采取其他措施：对于一张图片的原图/增强图片混合，但这样一定会增加数据集的大小

In [5]:
'''
训练集转换层，这些变换太多了，具体的变换见torch官网文档中的Examples and training references
并且对于RandomChoice，Compose，RandomApply第一个参数都是可执行变换的参数列表，这些函数都是返回
可执行变换的流水线列表，所以他们之间可以进行嵌套并自定义转换模型对数据进行处理
'''

train_tfm = v2.Compose([
    v2.ToImage(),
    v2.Resize((128,128)),
    #随机选择一个变换将其作用于图像，第二个参数p可以指定一个列表对应序列中变换被选择出的概率(自动标准化)
    v2.RandomApply([
        v2.RandomChoice([
            v2.ColorJitter(brightness=0.4, contrast=0.2, hue =0.3), #随机改变图像的亮度、对比度、饱和度和hue
            v2.RandomRotation(degrees=(0, 30)),                     #随机对图像在给定范围内正负rotation
            v2.RandomInvert(p=1.0),                                 #按指定概率对图像颜色进行反转
            v2.RandomSolarize(192, p=1.0),                          #按阈值对图像颜色进行反转
            v2.RandomAutocontrast(p=1.0),                           #调整对比度
            v2.RandomAdjustSharpness(2.0, p=1.0),                   #锐化
            v2.RandomHorizontalFlip(p=1.0),                         #水平翻转
        ])
    ], p=0.35),
    v2.ToDtype(torch.float32, scale=True)                           #scale是否将其值缩放到[0,1]之间即标准化
])

#通常我们不需要任何在val阶段和test阶段的augmentation，只需要resize再转化为tensor即可
#但我们可以使用train_tfm来处理测试集来产生不同类型的图片，但使用集成方法来决定test的结果
test_tfm = v2.Compose([
    v2.ToImage(),
    v2.Resize(128, 128),
    v2.ToDtype(torch.float32, scale=True)
])

#无需将测试数据转换为张量，如果这么做了就要在测试函数中还原，比较麻烦
aug_tfm = v2.Compose([
    v2.ToImage(),
    v2.Resize(128, 128)
])

## Dataset
对于数据在模型输入之前的处理，有两种结构，第一种是将数据处理放在构造函数中，这样再getitem中便不用再做处理  
第二种则是定义好处理的流水线，例如transform层，在getitem中使用数据时再将数据进行变换。
<span style="font-size: 16px; letter-spacing: normal;">This is normal text.</span>


In [6]:
from pathlib import Path

class ImageDataset(Dataset):
    '''
    
    '''
    def __init__(self, mode, path, tfm=test_tfm, files=None):
        super(ImageDataset).__init__()
        self.transform = tfm
        self.mode = mode
        
        if mode == 'train' or mode == 'val':
            '''
            files:所有文件名的列表
            train_path:原先划分好的训练数据path，待融合
            val_path:原先划分好的val数据path，待融合
            分别提取出train文件夹中的数据和validation文件夹中的文件名list，并merge为file便于交叉验证和其他处理
            由于我们既然将两个文件夹中的图片融合了，那么他们之间就是等价的。无论怎么拼接都是可行的
            '''
            train_path, val_path = os.path.join(path,'training'), os.path.join(path,'validation')
            files_train = sorted([str(Path(path) / 'training' / x) for x in os.listdir(train_path) if x.endswith(".jpg")])
            files_val = sorted([str(Path(path) / 'validation' / x) for x in os.listdir(val_path) if x.endswith(".jpg")])
            self.files = files_train + files_val
            #在windows系统下进行文件路径等处理的时候必须进行一些打印输出的操作，否则很容易被不同的文件路径格式
            print(self.files[:5])
        else:
            test_path = os.path.join(path,'test')
            self.files = sorted([str(Path(path) / 'test' / x) for x in os.listdir(test_path) if x.endswith('.jpg')])
        if files != None :
                self.files = files
        print(f"first of Image samples",self.files[0])
        
    def __len__(self):
        return len(self.files)
        
    def __getitem__(self, index):
        '''
        除了用transform对图像进行处理之外，还需要提取图像名中的标签。为文件名的开头第一个数字
        结构 Folder1/Floder2/0_1234.jpg 在这个路径中生成以 / 为分隔符的列表，再取列表最后一个
        元素，即'0_1234.jpg'进行处理,同理再进行一次 _ 分割就可以得到标签。

        即使测试集用不到标签，我们也对其处理以下
        '''
        fname = self.files[index]
        image = Image.open(fname)
        if(image == None): print('None image while getitem fname:',fname)
            
        if self.mode == 'train' or self.mode == 'val': 
            image = self.transform(image)
            try:
                label = int(fname.split('\\')[-1].split('_')[0])
                assert label >= 0 and label <= 10 ,"Labels are out of bounds"
            except:
                label = -1
        else:
            image = self.transform(image)
            label = -1
        return image, label

    
    def generate_augmentation_testfilelist(self, fname):
        '''
        fname:测试图片的文件名/路径
        生成测试图片的所有增强并返回包含原图片的所有增强的图片列表
        return:imagelist:从原图片开始按顺序排列的增强图片列表
        '''
        imagelist = []
        original = Image.open(fname)
        original = v2.ToImage(original)
        original = F.resize(original, 128, 128)
        imagelist.append(orignal)
        
        v2.ToDtype(torch.float32, scale=True)
        picture = F.to_dtype(picure, torch.float32, scale=True)
        
        augmentations = [
            v2.ColorJitter(brightness=0.4, contrast=0.2, hue =0.3), #随机改变图像的亮度、对比度、饱和度和hue
            v2.RandomRotation(degrees=(0, 30)),                     #随机对图像在给定范围内正负rotation
            v2.RandomInvert(p=1.0),                                 #按指定概率对图像颜色进行反转
            v2.RandomSolarize(192, p=1.0),                          #按阈值对图像颜色进行反转
            v2.RandomAutocontrast(p=1.0),                           #调整对比度
            v2.RandomAdjustSharpness(2.0, p=1.0),                   #锐化
            v2.RandomHorizontalFlip(p=1.0),                         #水平翻转
        ]
        for i, transform in enumerate(augmentations):
            transformed_image = transform(original)
            transformed_image = F.to_dtype(transformed_image, torch.float32, scale = True)
            imagelist.append(transformed_image)
        return imagelist 

    def get_files(self):
        '''
        返回所有文件路径列表使得KFold可根据文件名划分
        '''
        return self.files

## Model
由于计算量太大，所以要求实现的网络比较浅而且并不是每一个连接处都有shortcut，而是隔一个才有一个

由于resnet论文中并没有详细说明实现，只能看源码来学习数据流动了。经过查看源码对于原文中较浅的resnet来说
未使用bottleNeck结构的结构如下：
* 每次跳跃两层，并且identity与Residual总是在进入卷积层之前分离，在经过两层卷积层之后ReLU之前汇合，对于主路线有如下数据流动方式
* Residual与identity分离 Residual进入->Conv->BatchNorm->ReLU->Conv->Batch-> Residual + identity ->ReLU
从下方的源码和原论文中的模型结构可以看出只有中间四个模块化主要层是进行了分离处理的，开头7 * 7的卷积核处理、batchNorm
ReLU和pool都是将残差和identity并不分离一起处理的，这一点在代码实现中需要注意。在经过四个主要卷积模块处理之后
最后也是将数据都汇入主干进行进行平均池化和全连接层的处理，全连接层设置在softmax之前是必要的


```python
    self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding
        =1)
        # 通过_make_layer带到层次化设，只有中间这四行中包含shortcut实现计的效果
        self.layer1 = self._make_layer(block, 64, layers[0])  # 对应着conv2_x
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])  # 对应着conv3_x
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])  # 对应着conv4_x
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])  # 对应着conv5_x
        # 分类头
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expan
### 步幅为1的卷积和步幅为2的最大池化还是直接步幅为2的下采样？
* 卷积 + 步长为2最大池化：
在很多网络架构中，如 VGG，卷积层后跟最大池化是一种常见的设计。它可以在保持特征表达能力的同时减少计算量。
这种设计可以使网络更具稳定性，因为最大池化减少了特征图的空间尺寸，并且有助于防止过拟合。

* 步长为 2 的卷积(SownSampling)：
在许多现代卷积神经网络（如 ResNet 和 DenseNet）中，步长为 2 的卷积是常见的下采样方法。这种方法可以更有效地融合特征，因为它在下采样的同时进行特征提取。
使用步长为 2 的卷积可以提高网络的表达能力，因为卷积操作能够整合更多的信息，从而可以表现得更好

<img src = 'structure.png' alt = 'structure.png' width = '600' position = 'center'>


### BasicBlock

In [7]:
# torch.nn.Conv2d(in_channels, out_channels, kernel_size(square), stride, padding)
# torch.nn.MaxPool2d(kernel_size, stride, padding)
class PrintLayer(nn.Module):
    def __init__(self,name):
        super(PrintLayer,self).__init__()
        self.name = name
    def forward(self, x):
        print(f'after {self.name} x.shape = {x.shape}')
        return x
        
class BasicBlock(nn.Module):
    # expansion: int = 1
    '''
    基础模块以及identity shortcut的实现
    '''
    def __init__(self, 
                 inplanes: int,
                 midplanes: int,
                 outplanes: int):
        
        super(BasicBlock, self).__init__()
        
        #self.cnn = nn.Sequential(
        #由于在实现Dataset的数据增强或是只是做transform处理的部分已经进行了标准化，所以在模型的初始阶段无需进行标准化
        self.conv1 = nn.Conv2d(inplanes, midplanes, kernel_size=3, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(midplanes)
        self.relu1 = nn.ReLU(inplace=True)#can optionally do the operation in-place. Default: False
        # self.pool1 = nn.MaxPool2d(3, 2, 0) 抛弃最大池化而采用下采样来降低维度
        
        self.conv2 = nn.Conv2d(midplanes, outplanes, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(outplanes)
        self.relu2 = nn.ReLU(inplace=True)
        # self.pool2 = nn.MaxPool2d(3, 2, 0)

        #下采样 用于处理shortcut传递identity与主线路大小不匹配的问题，下采样之后也是需要标准化处理的
        self.downSample = nn.Sequential(
            nn.Conv2d(inplanes, midplanes, kernel_size=1, stride=2,padding=0),
            # nn.Conv(midplanes, outplanes, kernel_size=1, stride=2)
            nn.BatchNorm2d(midplanes)
        )
        
    def forward(self, x):
        if x is None:
            raise ValueError("Input is None in BasicBlock forward method")
       
        # print(f"Input shape to BasicBlock: {x.shape}")#运行到这里就会出现 x 变成None的情况，但是在
       
        identity = x.clone()                          #残差和恒等映射分离
        residual = x
        residual = self.conv1(residual)
        residual = self.bn1(residual)
        # print(f'redsdiual shape after conv1 and bn1 :{residual.shape}')
        
        identity = self.downSample(identity)    #下采样以使得identity特征图的长宽和主线路上的长宽相同
        # print(f"identiy Shape after downSample: {identity.shape}")
        
        x = residual + identity                  #根据要求给出的网络结构残差只跳过一层卷积层就回到主线路
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        # print(f"x Shape after conv2 and bn2: {x.shape}")
        x = self.relu2(x)
        
        return x

### Classifier

In [8]:
 # torch.nn.Conv2d(in_channels, out_channels, kernel_size(square), stride, padding)
# torch.nn.MaxPool2d(kernel_size, stride, padding)
class Classifier(nn.Module):
    '''
        
    '''
    def __init__(self):
        super(Classifier, self).__init__()
        #做类似ResidualNet原论文中初始化时候的一个处理，最然我也不知道为什么，今天就这样吧 23：56/8-1
        self.pool = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=2, padding=3, bias=False), #64*64
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),                #32*32
        )
        self.cnn = nn.Sequential(
            BasicBlock(64, 64, 128),                                         #16*16
            # PrintLayer('1 block'),
            
            BasicBlock(128, 128, 128),                                       #8*8
            # PrintLayer('2 block'),
            
            BasicBlock(128, 256, 256),                                       #4*4
            # PrintLayer('3 block'),
            
            BasicBlock(256, 512, 512),                                       #2*2
            # PrintLayer('last block'),
        )
        #平均池化 + 
        self.fc = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),            #这一步很重要平均池化之后经过全连接层进行处理
            nn.Flatten(start_dim=1,end_dim=-1),      #在平均池化处理之后需要进行展平处理
            # PrintLayer('after Flatten size'),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 11),
        )
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, x):
        x = self.pool(x)
        x = self.cnn(x)
        
        x = self.fc(x)
        return x
    
    def cal_loss(self, preds, labels):
        return self.criterion(preds, labels)
        

## Train
* 对张量除了做类型变换之外的操作得到的也是张量，所以每次需要在最后进行转换.item() .numpy()转化为数值类型或是数组类型

In [9]:
from typing import Any
def train(train_set: DataLoader, 
          val_set: DataLoader, 
          model: nn.Module, 
          config: Any, 
          device: torch.device) -> None:
    '''
        通常train都需要五个参数以上
        train_set:training dataset(dataLoader)
        val_set:validation dataset(dataLoader)
        model:model instance
        config:hyperParameter
        device:compute device
    '''
    #-----------------------------training--------------------------------#
    model.train()
    optimizer = getattr(torch.optim,config['optimizer'])(
        model.parameters(),**config['optimizer_hParas'])
    n_epochs = config['n_epochs']
    epoch = 0
    best_acc = 0
    for epoch in range(n_epochs):
        train_batch_loss = []
        train_batch_accs = []
        for x, y in train_set:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            
            # if x == None :
            #     print('Rasie NoneTypeError:X is None')
            
            preds = model(x)
            
            assert preds.size(-1) == 11,\
            "Output dimension does not match the number of classes"
            # print(f'prediciton after epoch{epoch+1} {preds}')
            # print(f'label value:{y}')
            
            loss = model.cal_loss(preds, y)
            loss.backward()
            grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm = 10)
            optimizer.step()

            #Another method to calculate batchAcc,下面这种计算一个batch的acc的方式与HW2中的计算Acc的方法显然更好
            #比较简洁并且不容易出错，不需要考虑len(x)而是指即取平均值
            acc = (preds.argmax(dim=-1).detach() == y.detach()).float().mean()
            train_batch_accs.append(acc)
            
            loss_detached = loss.detach().cpu().item()      #detach loss
            train_batch_loss.append(loss_detached)
            
        train_loss = sum(train_batch_loss) / len(train_batch_loss)
        train_acc = sum(train_batch_accs) / len(train_batch_accs)
        print('[{:03d}/{:03d}] Train Acc：{:3.6f} Train Loss：{:3.6f}'.format(epoch+1, n_epochs, train_acc, train_loss))
        
        #-----------------------------validation--------------------------------#
        if len(val_set) > 0:
            val_acc, val_loss = validate(val_set, model, device)
            print('Val Acc：{:3.6f} Val Loss：{:3.6f}'.format(val_acc, val_loss))
    
            if val_acc > best_acc :
                best_acc = val_acc
                 # only save best to prevent output memory exceed error

                torch.save(model.state_dict(),config['save_path'])
                print('Saving model at epoch {:03d} Better Acc {:3.3f}'.format(epoch + 1, val_acc))
            
            else:
                print('Epoch {:03d} Not a better Acc {:3.3f}'.format(epoch + 1,val_acc))
        elif len(val_set) == 0:
            torch.save(model.state_dict(), config['save_path'])
            print('Saving model at last epoch')
            

## validation

In [10]:
def validate(val_set: DataLoader, model:nn.Module, device:torch.device):
    model.eval()
    val_epoch_acc = []
    val_epoch_loss = []
    for x, y in val_set:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            preds = model(x)
            loss = model.cal_loss(preds, y).detach().cpu().item()
        acc = (preds.argmax(dim=-1).detach() == y.detach()).float().mean()
        val_epoch_acc.append(acc)
        val_epoch_loss.append(loss)
    return sum(val_epoch_acc) / len(val_epoch_acc), sum(val_epoch_loss) / len(val_epoch_loss)

## Test
这里需要对测试数据进行增强后预测

### 一、不改动__getitem__()而在测试函数中定义变换并进行预测

```python
# 定义数据增强变换列表
augmentations = [
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.RandomVerticalFlip(p=1.0),
    transforms.RandomRotation(45),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1),
]

# 定义基本变换（无需增强时）
basic_transform = transforms.Compose([
    transforms.ToTensor(),
])

# 初始化数据集和数据加载器
test_dataset = ImageDataset('path/to/test/images', transform=basic_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# 测试函数，思想在于不是一次性生成所有增强图片而是一次处理一个，这里需要了解DataLoader的工作原理
def test(model, dataloader, device):
    model.eval()
    results = []
    with torch.no_grad():
        for images in dataloader:
            images = images.to(device)
            original_outputs = model(images)
            augmented_outputs = []
            
            for aug in augmentations:
                augmented_transform = transforms.Compose([aug, transforms.ToTensor()])
                #下面对图像处理的操作是已经经过getitem的变换
                augmented_image = aug(images.cpu().squeeze().permute(1, 2, 0))
                augmented_image = augmented_transform(augmented_image)
                augmented_image = augmented_image.unsqueeze(0).to(device)
                output = model(augmented_image)
                augmented_outputs.append(output)
            
            # Combine original and augmented outputs
            #可以自行改变加权方式
            all_outputs = torch.stack([original_outputs] + augmented_outputs)
            combined_output = torch.mean(all_outputs, dim=0)
            results.append(combined_output.cpu().numpy())
    
    return results
```

### 二、在__getitem__进行复杂预处理是不推荐的，会占用过多内存
DataLoader的内存占用情况与BatchSize和实现的Dataset中__init__()/__ getitem__()行为密切相关

In [11]:
def test(test_set: DataLoader, 
         model: nn.Module, 
         device: torch.device, 
         weight: list):
    '''
        测试函数，使用训练好的模型对需要预测的数据进行预测
        这里的test实现需要用到通过对增强过的数据加权来进行预测,通过Dataset的getitem来返回
        包含原图片的所有增强图片的列表，再
    '''
    model.eval()
    augmentation = [
        v2.ColorJitter(brightness=0.4, contrast=0.2, hue=0.3),
        v2.RandomRotation(degress=(0, 30)),
        v2.RandomInvert(p=1.0),
        v2.RandomSolarize(192, p=1.0),
        v2.RandomAutocontrast(p=1.0),
        v2.RandomAdjustSharpness(2.0, p=1.0),
        v2.RandomHorizontalFlip(p=1.0)
    ]

    basic_tfm = v2.Compose([v2.ToDtype(torch.float32, scale=True)])
    prediction = []                                   #存储一张图片的分类结果 
    preds = []                                        #存储所有图片的分类结果
    with torch.no_grad():
        for x, y in test_set:
            x, y = x.to(device), y.to(device)
            prediction.append(model(x).detach().cpu())              #原始图片的预测结果
            
            for i, aug in enumerate(augmentation):                  #枚举每一个变换
                transformed = aug(x)                                #增强变换
                transformed = basic_tfm(x)                          #基础变换，转换为张量
                prediction.append(model(x).detach().cpu())          #所有预测结果都存储在一个张量列表中
            
            weight = torch.tensor(weight)                           #权重列表
            weight_sum = sum(w*p for w, p in zip(weight,prediction))#通过列表表达式对每个图片的预测概率进行加权加和
           
            _,maxProbClass = tensor.max(weight,dim=0)               #找出最大概率对应的标签
            preds.append(maxProbClass)                              
        preds = torch.cat(preds,dim=0).numpy()                      #将概率对应标签进行拼接并转换为numpy类型
        
    return preds

def save_preds(preds, file):
    print('Saving results to {}'.format(file))
    with open(file, 'w') as fp:
        writer.writerow(['Id', 'Class'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])
            

## TrainingTime

In [12]:
device = get_device()

config = {
    'n_epochs':10,
    'batch_size':128,
    'optimizer':'Adam',
    'optimizer_hParas':{
        'lr':1e-3,
        # 'monmentum':0.9,
        # 'weight_decay':5e-4
    },
    'save_path':'models/model.pth',
    'n_folds':10,
    'weight':[0.5, 0.5/7, 0.5/7, 0.5/7, 0.5/7, 0.5/7, 0.5/7, 0.5/7]
}
path = './dataset/'

In [ ]:
from sklearn.model_selection import KFold
from torch.utils.data import SubsetRandomSampler

# 加载数据集
dataset = ImageDataset('train', path=path, tfm=train_tfm)
print(len(dataset))

# 初始化 KFold
kfold = KFold(n_splits=config['n_folds'], shuffle=True, random_state=myseed)

# 开始 K-fold 训练
for fold, (train_idx, val_idx) in enumerate(kfold.split(range(len(dataset)))):
    print(f'FOLD {fold+1}')
    print('---------------------------------------------------')
    
    # 每个折叠都重新初始化模型
    model = Classifier().to(device)
    
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    
    trainloader = DataLoader(dataset, batch_size=config['batch_size'], sampler=train_sampler)
    valloader = DataLoader(dataset, batch_size=config['batch_size'], sampler=val_sampler)
    
    # 开始训练
    train(train_set=trainloader, val_set=valloader, model=model, config=config, device=device)


['dataset\\training\\0_0.jpg', 'dataset\\training\\0_1.jpg', 'dataset\\training\\0_10.jpg', 'dataset\\training\\0_100.jpg', 'dataset\\training\\0_101.jpg']
first of Image samples dataset\training\0_0.jpg
13296
FOLD 1
---------------------------------------------------
[001/010] Train Acc：0.301143 Train Loss：1.996295
Val Acc：0.258551 Val Loss：2.389072
Saving model at epoch 001 Better Acc 0.259


## TestingTime

In [ ]:
# model_best = Classifier().to(device)
# model_best.load_state_dict(torch.load(config['save_path']))
# test_dataset = ImageDataset('test', path=path, tfm=aug_tfm)

# preds = test(test_dataset, models_best, device, config['weight'])
# save_preds(preds,'pred.csv')


